In [1]:
import os 
%pwd


'e:\\TextSummarization\\research'

In [2]:
os.chdir('../')
%pwd

'e:\\TextSummarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config
    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]

    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
                                    batch_size=16,device='cuda' if torch.cuda.is_available() else 'cpu',
                                    column_text='article',
                                    column_summary='highlights'
                                    ):
        article_batches=list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches=list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))

        for article_batch,target_batch in tqdm(
            zip(article_batches,target_batches),total=len(article_batches)
        ):
            inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding='max_length',return_tensors='pt')
            summaries=model.generate(inputs_id=inputs['input_ids'].to(device),
                                     attention_mask=inputs['attention_mask'].to(device),
                                     length_penalty=0.8,num_beams=8,max_length=128
                                     )
            decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries=[d.replace(""," ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries,references=target_batch)

        score=metric.compute()

        return score()
    
    def evaluate(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)

        model_t5=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        


